In [1]:
import numpy as np
import time

In [2]:
def _generate_random_matrices(matrix_dims,
                              seed: int=42):
  """Generates two random NumPy matrices to be multiplied."""
  np.random.seed(seed)
  a = np.random.randn(matrix_dims[0], matrix_dims[1])
  b = np.random.randn(matrix_dims[1], matrix_dims[2])
  return a, b


In [18]:
a,b=_generate_random_matrices((4*13,5*13,5*13))

In [19]:
a.shape

(52, 65)

In [20]:
b.shape

(65, 65)

In [21]:
def matmul(A,B):
  result=[[0 for i in range(B.shape[1])] for j in range(A.shape[0])]
  for i in range(len(A)):
    for j in range(len(B[0])):
        for k in range(len(B)):
          result[i][j] += A[i][k] * B[k][j]
  return np.array(result)

In [22]:
class mult:
  def __init__(self):
    filename = "/content/factorizations_r.npz"
    with open(filename, 'rb') as f:
      self.factorizations = dict(np.load(f, allow_pickle=True))
  def check(self,tup):
    far=None
    for v in self.factorizations:
      ar=list(map(int,v.split(",")))
      if tup[0]%ar[0]==0 and tup[1]%ar[1]==0 and tup[2]%ar[2]==0 :
        far=ar
    if far!=None:
      return True,far
    return False,None
  def comb(self,a):
    res=[]
    for i in range(a.shape[0]):
      ar=np.array(a[i][0])
      for j in range(1,a.shape[1]):
        ar=np.concatenate((ar,a[i][j]),axis=1)
      res.append(ar)
    nar=res[0]
    for i in range(1,len(res)):
      nar=np.concatenate((nar,res[i]),axis=0)
    return nar
  def crop(self,a,n):
    i=0
    j=0
    res=[[None for k in range(a.shape[1]//n)] for g in range(a.shape[0]//n)]
    x=0
    y=0
  # print(len(res),len(res[0]))
  #print(x,y,res[0][0],a.shape)
    while(i<a.shape[0]):
    # print(i,j,x,y)
       while(j<a.shape[1]):
         res[x][y]=a[i:i+n,j:j+n]
         j+=n
         y+=1
       x+=1
       i+=n
       j=0
       y=0
    
    return np.array(res)    
  def aft2(self,a,b,seq,f) :
    n=a.shape[0]//seq[0]
    val=str(seq[0])+","+str(seq[1])+","+str(seq[2])
    factors=self.factorizations[val]
    factors = [factors[0], factors[1], factors[2]]
    rank = factors[0].shape[-1]
    factors[2] = np.reshape(factors[2],(seq[2], seq[0], rank))
    factors[2] = factors[2].transpose(1, 0, 2).reshape(seq[2]*seq[0],rank)
    a=self.crop(a,n)
    b=self.crop(b,n)
    
    a=np.reshape(a,(a.shape[0]*a.shape[1],n,n))
    b=np.reshape(b,(b.shape[0]*b.shape[1],n,n))

    st=time.time()
    left=np.einsum('iR,i...->R...', factors[0], a)
    right=np.einsum('iR,i...->R...', factors[1], b)
    mp=np.einsum('R...,R...->R...', left, right)
    mp=np.array([f(left[i],right[i] )for i in range(len(left))])
    res=np.einsum('iR,R...->i...', factors[2], mp)
    en=time.time()
    res=np.reshape(res,(seq[0],seq[2],n,n))
    print(en-st)
    # print(res.shape)
    return self.comb(res)
    
    
  
  

In [23]:
mul=mult()

In [24]:
val,seq=mul.check([a.shape[0],a.shape[1],b.shape[1]])

In [25]:
seq

[4, 5, 5]

In [26]:
st=time.time()
c=mul.aft2(a,b,[4,5,5],matmul)
en=time.time()
print(en-st)

0.10695743560791016
0.10853838920593262


In [27]:
st=time.time()
c1=matmul(a,b)
en=time.time()
print(en-st)

0.12349486351013184


In [28]:
np.isclose(c,c1)

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])